In [2]:
import csv
import os
from datetime import datetime
from fastapi import Request
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from preprocessing import *


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajeev.mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
app = FastAPI()

model=load_model(r"model/cleandata_basic_30L_2.h5")
multilabel_binarizer = joblib.load(r"classes/cleandata_Updated_mlb_All_Product_2.pkl")
labels = multilabel_binarizer.classes_
tokenizer = joblib.load(r"tokenizer/cleandata_Updated_tokenizer_30L_2.pkl")
full_form_data=pd.read_csv(r"shortCodesProduct.csv")
full_form_data=full_form_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:

def log_predictions(input_sentence, predictions):
    log_file = "prediction_log.csv"
    file_exists = os.path.exists(log_file)

    with open(log_file, mode="a", newline="") as file:
        writer = csv.writer(file)

        if not file_exists:
            writer.writerow(["Date Time", "IP Address", "Input Sentence", "Predicted Labels"])

        date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # ip_address = request.client.host

        writer.writerow([date_time, input_sentence, predictions])

def predict_op_th_main(new_sentence, threshold):
    new_sentence=new_sentence.lower()
    new_sentence=replace_short_with_full(new_sentence, full_form_data)
    new_sentence = cleanText(new_sentence)
    new_sentence = preprocess_text(new_sentence)
    new_sentence_sequences = tokenizer.texts_to_sequences([new_sentence])
    new_sentence_padded = pad_sequences(new_sentence_sequences, maxlen=512)
    predicted_domain = model.predict(new_sentence_padded)
    # result = [{label,  float(prob)} for label, prob in zip(labels, predicted_domain[0]) if prob >= threshold]
    result = [{label} for label, prob in zip(labels, predicted_domain[0]) if prob >= threshold]

    print(type(result))
    return result

def Predict_Labels(new_sentence):
    threshold = 0.3
    output = predict_op_th_main(new_sentence, threshold)
    if not output:
        threshold = 0.15
        output = predict_op_th_main(new_sentence, threshold)
    return output



In [5]:
Predict_Labels("supply of tata punch")

1/1 [==============================] - 0s 256ms/step
<class 'list'>
1/1 [==============================] - 0s 11ms/step
<class 'list'>


[{'automobile ancillaries'}, {'tool'}, {'vehicle'}]

In [6]:
import numpy as np
import tensorflow as tf
import shap
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
from nltk.corpus import stopwords, 

In [11]:
import shap

# Load your machine learning model (e.g., replace with your actual model loading code)


# Check if the model is supported by SHAP for explanation
is_supported = shap.supports_model(model)

if is_supported:
    print("Model is supported by SHAP for explanation.")
else:
    print("Model is not supported by SHAP for explanation.")


AttributeError: module 'shap' has no attribute 'supports_model'

In [32]:
def predict_sentence(sentence):
    output = Predict_Labels(sentence)
    log_predictions(sentence, output)
    return output

In [33]:
predict_sentence("purani nagarpalika shopping complex dukan kramank 06 any pichda varg aarakchit")

1/1 [==============================] - 0s 14ms/step
<class 'list'>
1/1 [==============================] - 0s 12ms/step
<class 'list'>


[{'building'}, {'civil work'}]

In [35]:
# def predict_sentence(request: Request, data: InputData):
def predict_sentence(data: InputData):
    output = Predict_Labels(data.sentence)
    log_predictions(data.sentence, output)
    return output

In [38]:
class InputData(BaseModel):
    sentence: str

@app.post("/prediction3")
def predict_sentence(request: Request, data: InputData):
    output = Predict_Labels(data.sentence)
    log_predictions(request, data.sentence, output)
    return output



In [47]:
df2.to_csv('prediction_global_data_top10000.csv')

In [43]:
import pandas as pd
df2 = pd.read_excel('GloalTendersPrediction.xlsx')
print(df2)

          srno      tcno                                       tendersBrief  \
0     70164532  59513614  Digitization Of The "wille" Memorial, Lot 1: P...   
1     70164533  59513615  Procurement Of Office Supplies For Periodical ...   
2     70164534  59513616  Awarding Of Scholarships For Professionally Qu...   
3     70164535  59513617  Supply And Delivery Of Materials For Construct...   
4     70164536  59513618  Demolition Work - Partial Demolition Of Gollno...   
...        ...       ...                                                ...   
9853  53344712  51097972      Provision For Procurement Of Air Conditioning   
9854  53344401  51097661  Delivery Of Servers And Provision Of Cable Con...   
9855  53344408  51097668  Services Of Plugging, Drilling, Re-drilling An...   
9856  53344410  51097670  Delivery And Maintenance Of Network Equipment ...   
9857  53344411  51097671  Consulting Services For The Assessment Of The ...   

                                            product

In [ ]:

df2['predictions'] = df2['tendersBrief'].apply(Predict_Labels)

In [48]:
curl -X POST "http://192.168.50.247:8001/prediction3" -H "accept: application/json" -H "Content-Type: application/json" -d '{"text": "purani nagarpalika shopping complex dukan kramank 06 any pichda varg aarakchit"}'

SyntaxError: invalid syntax (1655069604.py, line 1)

# Interpreting EXPLAINABLE AI

In [53]:
import numpy as np
import tensorflow as tf
import shap
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Load pre-trained deep learning text classification model (e.g., TensorFlow/Keras model)
model = tf.keras.models.load_model(r'model/cleandata_basic_30L_2.h5')

In [ ]:


# Load necessary files for interpretation (e.g., vocabulary, tokenizer, etc.)
# Replace with actual paths and loading methods for your use case
vocabulary = np.load('vocabulary.npy', allow_pickle=True).item()
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=len(vocabulary))
tokenizer.word_index = vocabulary

# Load NLTK stopwords for text preprocessing
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load SHAP explainer
explainer = shap.Explainer(model, tokenizer)

# Example sentence for interpretation
sentence = "This is a great product that works well."

# Preprocess the sentence for prediction and interpretation
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

preprocessed_sentence = preprocess_text(sentence)
input_data = tokenizer.texts_to_sequences([preprocessed_sentence])
input_data = tf.keras.preprocessing.sequence.pad_sequences(input_data, maxlen=50)

# Get SHAP explanations for the prediction
shap_values = explainer([input_data])

# Summarize the impact of words/tokens on the prediction
print("SHAP Values for Each Token:")
for token, shap_value in zip(preprocessed_sentence.split(), shap_values[0][0]):
    print(f"Token: {token}, SHAP Value: {shap_value}")


In [55]:
import pickle
import numpy as np
import tensorflow as tf
import shap
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Load pre-trained deep learning text classification model (e.g., TensorFlow/Keras model)
model = tf.keras.models.load_model(r'model/cleandata_basic_30L_2.h5')

# Load tokenizer
with open(r'tokenizer/cleandata_Updated_tokenizer_30L_2.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

# Load MultiLabelBinarizer classes
with open(r'classes/cleandata_Updated_mlb_All_Product_2.pkl', 'rb') as mlb_classes_file:
    mlb_classes = pickle.load(mlb_classes_file)

In [56]:
# Load NLTK stopwords for text preprocessing
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load SHAP explainer
explainer = shap.Explainer(model, tokenizer)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajeev.mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
sentence = "This product is well designed and easy to use."

In [58]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

In [59]:
preprocessed_sentence = preprocess_text(sentence)
input_data = tokenizer.texts_to_sequences([preprocessed_sentence])
input_data = tf.keras.preprocessing.sequence.pad_sequences(input_data, maxlen=50)

In [1]:
# Load NLTK stopwords for text preprocessing
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load SHAP explainer
explainer = shap.Explainer(model, tokenizer)

# Example sentence for interpretation
sentence = "This product is well designed and easy to use."

# Preprocess the sentence for prediction and interpretation
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

preprocessed_sentence = preprocess_text(sentence)
input_data = tokenizer.texts_to_sequences([preprocessed_sentence])
input_data = tf.keras.preprocessing.sequence.pad_sequences(input_data, maxlen=50)

# Get SHAP explanations for the prediction
shap_values = explainer([input_data])

# Summarize the impact of words/tokens on the prediction
print("SHAP Values for Each Token:")
for token, shap_value in zip(preprocessed_sentence.split(), shap_values[0][0]):
    print(f"Token: {token}, SHAP Value: {shap_value}")


NameError: name 'nltk' is not defined

In [71]:
dir(shap.Explainer)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_compute_main_effects',
 '_instantiated_load',
 'explain_row',
 'load',
 'save',
 'supports_model_with_masker']

In [69]:
def predict_text(sentence: str, threshold: float = 0.1):
    result = predict_op_th_main(sentence, threshold)
    
    # Initialize SHAP explainer
    explainer = shap.Explainer(predict_op_th_main, tokenizer, max_evals = 2000)
    
    # Preprocess the sentence for SHAP interpretation
    preprocessed_sentence = preprocess_text(sentence)
    input_data = tokenizer.texts_to_sequences([preprocessed_sentence])
    input_data_padded = pad_sequences(input_data, maxlen=512)
    
    # Calculate SHAP values
    shap_values = explainer(input_data_padded)
    
    # Summarize SHAP values for each label
    shap_summary = {}
    for idx, label in enumerate(labels):
        shap_values_label = shap_values[0][idx]
        shap_summary[label] = shap_values_label.tolist()
    
    return {"predictions": result, "shap_summary": shap_summary}

In [70]:
predict_text("construction of road")

1/1 [==============================] - 0s 14ms/step
<class 'list'>


TypeError: 'Tokenizer' object is not callable